In [10]:
!pip install chess

In [11]:
import pandas as pd
import numpy as np
from tensorflow.keras import layers, models, Input
import tensorflow as tf

In [12]:
print("TensorFlow version:", tf.__version__)
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))


TensorFlow version: 2.19.0
Num GPUs Available: 1


In [13]:
data = pd.read_csv("fens_training_set.csv")
data.head()

,fen,move
0,7k/5p1p/p2p1Pr1/1p4pQ/8/P1P5/2pr3P/2R2K2 w - -...,h5f3
1,5r2/2p3k1/1p3pb1/p1pPr2p/2P1PRP1/3B1R2/PP4K1/8...,g4h5
2,r2r2k1/p3bpp1/q2Bp2p/2pn4/2b1N1Q1/8/PPP2PPP/3R...,d6e7
3,8/8/8/7K/1k6/8/6P1/8 w - - 0 1,g2g4
4,rnbqkb1r/pp3ppp/5n2/3p4/2pP4/3B1N2/PPP2PPP/RNB...,f1e1


In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 268550 entries, 0 to 268549
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   fen     268550 non-null  object
 1   move    268550 non-null  object
dtypes: object(2)
memory usage: 4.1+ MB


In [15]:
print(data.loc[1, 'fen'])
print(data.loc[0, 'move'])

5r2/2p3k1/1p3pb1/p1pPr2p/2P1PRP1/3B1R2/PP4K1/8 w - - 0 1
h5f3


In [16]:
import chess

board = chess.Board('5r2/2p3k1/1p3pb1/p1pPr2p/2P1PRP1/3B1R2/PP4K1/8 w - - 0 1')
print(board)


. . . . . r . .
. . p . . . k .
. p . . . p b .
p . p P r . . p
. . P . P R P .
. . . B . R . .
P P . . . . K .
. . . . . . . .


In [17]:
board.piece_map()

{61: Piece.from_symbol('r'),
 54: Piece.from_symbol('k'),
 50: Piece.from_symbol('p'),
 46: Piece.from_symbol('b'),
 45: Piece.from_symbol('p'),
 41: Piece.from_symbol('p'),
 39: Piece.from_symbol('p'),
 36: Piece.from_symbol('r'),
 35: Piece.from_symbol('P'),
 34: Piece.from_symbol('p'),
 32: Piece.from_symbol('p'),
 30: Piece.from_symbol('P'),
 29: Piece.from_symbol('R'),
 28: Piece.from_symbol('P'),
 26: Piece.from_symbol('P'),
 21: Piece.from_symbol('R'),
 19: Piece.from_symbol('B'),
 14: Piece.from_symbol('K'),
 9: Piece.from_symbol('P'),
 8: Piece.from_symbol('P')}

In [18]:
for square, piece in board.piece_map().items():
    print(square, piece)

61 r
54 k
50 p
46 b
45 p
41 p
39 p
36 r
35 P
34 p
32 p
30 P
29 R
28 P
26 P
21 R
19 B
14 K
9 P
8 P


In [19]:
piece_to_index = {
    'P': 0, 'N': 1, 'B': 2, 'R': 3, 'Q': 4, 'K': 5,
    'p': 6, 'n': 7, 'b': 8, 'r': 9, 'q': 10, 'k': 11
}

In [20]:
matrix = np.zeros((8, 8, 12), dtype=np.int8)
for square, piece in board.piece_map().items():
    row = 7 - (square // 8)
    col = square % 8
    matrix[row, col, piece_to_index[piece.symbol()]] = 1

In [21]:
def fen_to_features(fen):
    piece_to_index = {
        'P': 0, 'N': 1, 'B': 2, 'R': 3, 'Q': 4, 'K': 5,
        'p': 6, 'n': 7, 'b': 8, 'r': 9, 'q': 10, 'k': 11
    }

    board = chess.Board(fen)
    matrix = np.zeros((8, 8, 12), dtype=np.int8)

    for square, piece in board.piece_map().items():
        row = 7 - (square // 8)
        col = square % 8
        matrix[row, col, piece_to_index[piece.symbol()]] = 1

    extra_features = []
    extra_features.append(1 if board.turn == chess.WHITE else 0)

    extra_features.append(1 if board.has_kingside_castling_rights(chess.WHITE) else 0)
    extra_features.append(1 if board.has_queenside_castling_rights(chess.WHITE) else 0)
    extra_features.append(1 if board.has_kingside_castling_rights(chess.BLACK) else 0)
    extra_features.append(1 if board.has_queenside_castling_rights(chess.BLACK) else 0)

    en_passant = np.zeros(8, dtype= np.int8)
    if board.ep_square is not None:
        col = board.ep_square % 8
        en_passant[col] = 1

    extra_features.append(board.halfmove_clock / 50.0)
    extra_features.append(board.fullmove_number / 100.0)

    extra_features = np.concatenate([np.array(extra_features, dtype=np.float32), en_passant])
    return matrix, extra_features

In [22]:
X_board = []
X_meta = []

for fen in data["fen"]:
    board_mat, extra = fen_to_features(fen)
    X_board.append(board_mat)
    X_meta.append(extra)

X_board = np.stack(X_board)
X_meta = np.stack(X_meta)

In [23]:
def move_to_indices(move_uci):
    move = chess.Move.from_uci(move_uci)
    return move.from_square, move.to_square

In [24]:
y_from = np.array([move_to_indices(m)[0] for m in data["move"]])
y_to = np.array([move_to_indices(m)[1] for m in data["move"]])

In [ ]:
board_input = Input(shape=(8, 8, 12))
meta_input = Input(shape=(15, ))
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(board_input)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(x)
x = layers.Flatten()(x)

y = layers.Dense(64, activation='relu')(meta_input)

z = layers.concatenate([x, y])
z = layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(z)
z = layers.Dropout(0.5)(z)

from_output = layers.Dense(64, activation='softmax', name="from_square")(z)
to_output = layers.Dense(64, activation='softmax', name="to_square")(z)

model = models.Model(inputs=[board_input, meta_input], outputs=[from_output, to_output])
model.compile(
    optimizer='adam',
    loss=['sparse_categorical_crossentropy', 'sparse_categorical_crossentropy'],
    metrics=['accuracy', 'accuracy']
)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 8, 8, 12)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 8, 8, 64)  │      6,976 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 8, 8, 64)  │     36,928 │ conv2d[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 15)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 4096)      │          0 │ conv2d_1[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │      1,024 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 4160)      │          0 │ flatten[0][0],    │
│ (Concatenate)       │                   │            │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 256)       │  1,065,216 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 256)       │          0 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ from_square (Dense) │ (None, 64)        │     16,448 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ to_square (Dense)   │ (None, 64)        │     16,448 │ dropout[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,143,040 (4.36 MB)

 Trainable params: 1,143,040 (4.36 MB)

 Non-trainable params: 0 (0.00 B)

In [26]:
history = model.fit(
    [X_board, X_meta],
    [y_from, y_to],
    validation_split=0.1,
    batch_size=64,
    epochs=20,
    verbose=1
)


Epoch 1/20
3777/3777 ━━━━━━━━━━━━━━━━━━━━ 24s 5ms/step - from_square_accuracy: 0.3252 - from_square_loss: 2.3415 - loss: 5.7716 - to_square_accuracy: 0.1465 - to_square_loss: 3.4300 - val_from_square_accuracy: 0.4757 - val_from_square_loss: 1.5591 - val_loss: 4.3005 - val_to_square_accuracy: 0.2779 - val_to_square_loss: 2.7415
Epoch 2/20
3777/3777 ━━━━━━━━━━━━━━━━━━━━ 15s 4ms/step - from_square_accuracy: 0.4728 - from_square_loss: 1.6134 - loss: 4.3874 - to_square_accuracy: 0.2722 - to_square_loss: 2.7741 - val_from_square_accuracy: 0.4833 - val_from_square_loss: 1.4871 - val_loss: 4.0900 - val_to_square_accuracy: 0.3002 - val_to_square_loss: 2.6029
Epoch 3/20
3777/3777 ━━━━━━━━━━━━━━━━━━━━ 15s 4ms/step - from_square_accuracy: 0.5068 - from_square_loss: 1.4806 - loss: 4.0658 - to_square_accuracy: 0.3111 - to_square_loss: 2.5851 - val_from_square_accuracy: 0.4964 - val_from_square_loss: 1.4644 - val_loss: 3.9903 - val_to_square_accuracy: 0.3147 - val_to_square_loss: 2.5261
Epoch 4/20
37

In [28]:
model.save("chess_move_model.keras")
model.save_weights("chess_move.weights.h5")